<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Text_Generation_using_Fnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install libraries and import

In [2]:
!pip install datasets
!pip install torch[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00


In [3]:
import torch

## 2. Loading Data

In [4]:
from datasets import load_dataset
datasets = load_dataset('wikitext','wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [5]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

## 3. PREPROCESSING DATA

In [6]:
import re
def preprocess_text(sentence):
  text = sentence['text'].lower() # lowering the sentence and storing in text vaiable
  text = re.sub('[^a-z?!.,]',' ',text) # removing other than characters and punctuations
  text = re.sub('\s\s+',' ',text) # removing double spaces
  sentence['text'] = text
  return sentence

In [7]:
datasets['train'] = datasets['train'].map(preprocess_text)
datasets['test'] = datasets['test'].map(preprocess_text)
datasets['validation'] = datasets['validation'].map(preprocess_text)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
datasets['train'] = datasets['train'].filter(lambda x : len(x['text']) > 20)
datasets['test'] = datasets['test'].filter(lambda x : len(x['text']) > 20)
datasets['validation'] = datasets['validation'].filter(lambda x : len(x['text']) > 20)

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [9]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2312
    })
    train: Dataset({
        features: ['text'],
        num_rows: 18794
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1988
    })
})

## 3. TOKENIZATION

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [12]:
vocab_size = len(tokenizer.vocab)

In [13]:
def tokenize(sentence):
  sentence = tokenizer(sentence['text'],truncation = True)

  return sentence

tokenized_inputs = datasets['train'].map(tokenize)

Map:   0%|          | 0/18794 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

batch = 16

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
dataloader = DataLoader(list(zip(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])),batch_size=batch,
                        collate_fn=data_collator)

## 4. MODEL

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft as fft

In [49]:
class FNetEncoder(nn.Module):

  def __init__(self,embed_dim, dense_dim):
    super(FNetEncoder,self).__init__()
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
#    self.num_heads = num_heads


   # self.positional_embedding = nn.Embedding(max_seq_len, embed_dim)
    #self.embedding = nn.Embedding(vocab_size,embed_dim)
    self.dense_proj = nn.Sequential(nn.Linear(self.embed_dim,self.dense_dim), nn.ReLU(), nn.Linear(self.dense_dim,self.embed_dim))

    self.layernorm_1 = nn.LayerNorm(self.embed_dim)
    self.layernorm_2 = nn.LayerNorm(self.embed_dim)

  def forward(self,inputs):
    # embeddign the inputs
    #embedded_inputs = self.embedding(inputs)

    # # Adding positional embeddings
    # seq_length = inputs.size(1)
    # positions = torch.arange(0, seq_length, device=inputs.device).unsqueeze(0)
    # positional_embeddings = self.positional_embedding(positions)

    # # Summing token embeddings and positional embeddings
    # embedded_inputs = embedded_inputs + positional_embeddings

    #castinn the inputs to compex
   # inp_complex = torch.view_as_complex(inputs)

    # getting fourier transform
    fft_result = fft.fft2(inputs)

    #taking real part
    fft_real = fft_result.real.float()

    proj_input = self.layernorm_1 (inputs + fft_real)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input +proj_output)





In [50]:
class FNetTextGenerator(nn.Module):
    def __init__(self, embed_dim, latent_dim, vocab_size, max_seq_len, num_heads, num_layers):
        super(FNetTextGenerator, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.positional_embedding = nn.Embedding(max_seq_len, embed_dim)
        self.transformer_decoder_layer = nn.TransformerDecoderLayer(embed_dim, num_heads)
        self.transformer_decoder = nn.TransformerDecoder(self.transformer_decoder_layer, num_layers=num_layers)

        self.transformer_encoder_layer = FNetEncoder(embed_dim, latent_dim)
      #  self.transformer_encoder = torch.nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = num_layers)

        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, inputs, target=None):
        seq_length = inputs.size(1)

        embedded_tokens = self.embedding(inputs)
        positions = torch.arange(0, seq_length, device=inputs.device).unsqueeze(0)
        embedded_positions = self.positional_embedding(positions)
        encoder_input = embedded_tokens + embedded_positions

        encoder_output = self.transformer_encoder_layer(encoder_input)

        if target is not None:
            target = target.permute(1, 0, 2)  # Adjust the shape for transformer decoder
            decoder_output = self.transformer_decoder(target, encoder_output)
            return decoder_output.permute(1, 0, 2)  # Adjust the shape back to batch-first

        # If no target is provided, generate autoregressively
        batch_size, seq_len = inputs.size()
        generated_sequence = torch.zeros(batch_size, seq_len, dtype=torch.long, device=inputs.device)

        # Initial input for autoregressive decoding
        input_token = inputs[:, 0].unsqueeze(1)

        for t in range(1, seq_len):
            embedded_input_token = self.embedding(input_token) + self.positional_embedding(torch.tensor([[t]]).to(inputs.device))
            decoder_output = self.transformer_decoder(embedded_input_token, encoder_output)
            logits = self.fc(decoder_output[-1])
            _, predicted_token = torch.max(F.softmax(logits, dim=-1), dim=-1)
            generated_sequence[:, t] = predicted_token.squeeze()

            input_token = predicted_token.unsqueeze(1)

        return generated_sequence



In [51]:
# Example usage
embed_dim = 256
latent_dim = 512
vocab_size = 10000
max_seq_len = 50
num_heads = 8
num_layers = 6

model = FNetTextGenerator(embed_dim, latent_dim, vocab_size, max_seq_len, num_heads, num_layers)

# Dummy input
inputs = torch.randint(0, vocab_size, (2, max_seq_len))

# Forward pass for text generation
generated_sequence = model(inputs)
print("Generated Sequence:", generated_sequence)

RuntimeError: ignored